In [3]:
import serial
import time
import pandas as pd

import scipy
from scipy import signal
import numpy as np

import math
from statistics import mean

In [30]:
import pickle
global clf
clf = pickle.load(open('testmodel.pkl','rb'))

In [37]:
SIG_SIZE = 90 # 1 sec contains 30 samples ; 3 seconds 90 samples.
THRSHLD = 30

BUFFER_SIZE = 360
global COUNT
COUNT = 0
buffer = np.array([])
time_buffer = np.array([])
test_sig_db = np.array([])
test_sig_ts_db = []
i = 1
prev_mean = 0

In [7]:
def event_detection(data):
    data = scipy.signal.medfilt(data, kernel_size=None)
    dbydx = np.gradient(data)
    on_events = []
    for idx,x in enumerate(dbydx):
        if x > THRSHLD:
            on_events.append(idx)
    for x in on_events:
        for y in range(x+1,x+91): # to remove false turn on signatures, give delay for transition
            if y in on_events:
                on_events.remove(y)
    return on_events

In [38]:
def get_signal (length):
    '''
    Takes the length of signature required,
    returns signature and timestamps
    
    '''
    global COUNT
    buffer = np.array([])
    time_buffer = np.array([])
    while COUNT!= length :
        b = ser.readline()         # read a byte string
        string_n = b.decode(errors='ignore')  # decode byte string into Unicode  
        print(string_n)
        try:
            val, timestamp = string_n.split(sep = ',')
            val = float(val)
            timestamp = pd.to_datetime(timestamp)
            buffer = np.append(buffer, val) # helps getting data in seconds
            time_buffer = np.append(time_buffer,timestamp)
            COUNT += 1
        except:
            print('read failed, retrying!')
    COUNT = 0
    return buffer,time_buffer

In [35]:
def predict_clf(data): #unused
    global clf
    guess = clf.predict(data)
    acc = clf.predict_proba(data).max()
    print(f'Device turned on at {sig_ts} and is probably {guess}, Acc.: {acc}') 

In [23]:
# set up the serial line
ser = serial.Serial('COM3', 115200)

time.sleep(2)

In [39]:
# Read and record the data
for i in range(10):
    buffer, time_buffer = get_signal(BUFFER_SIZE)
    events = event_detection(buffer)
    if len(events)>0:
        print(f'{len(events)} events detected')
        for ev in events:
            sig = buffer[ev:ev+SIG_SIZE]
            if len(sig) != SIG_SIZE:
                print('signal not adequate, shifting buffer')
                buffer = np.roll(buffer, -SIG_SIZE)
                time_buffer = np.roll(time_buffer, -SIG_SIZE)
                temp_buffer, temp_time_buffer = get_signal(SIG_SIZE)
                buffer[BUFFER_SIZE-SIG_SIZE:] = temp_buffer
                time_buffer[BUFFER_SIZE-SIG_SIZE:] = temp_time_buffer
                events = event_detection(buffer)
                if len(events)>0:
                    print(f'{len(events)} events detected on reconstructing')
                    for ev in events:
                        sig = buffer[ev:ev+SIG_SIZE]
                        if sig.mean() > prev_mean:
                            sig = sig - prev_mean
                            sig_ts = time_buffer[ev]
                            test_sig_db = np.append(test_sig_db, sig, axis = 0)
                            test_sig_ts_db = np.append(test_sig_ts_db,sig_ts)
                            test_sig_db = test_sig_db.reshape((test_sig_db.shape[0]//SIG_SIZE,SIG_SIZE))
                            rise = buffer[ev+SIG_SIZE:].mean()
                            guess = clf.predict(test_sig_db)
                            acc = clf.predict_proba(test_sig_db).max()
                            print(f'{guess} turned on at {sig_ts} with rise {rise}, Acc.: {acc}')
                            test_sig_db = np.array([])
                            test_sig_ts_db = np.array([])
                            buffer = np.array([])
                            time_buffer = np.array([])
                    break # breaks previous events for loops 
        break #breaks i iterator
                
#                 if not any(ele >= (I-90) and ele < I for ele in test_sig_ts_events):
#                     if I not in idx_on_events:
#                         idx_on_events.append(I)
#                         test_vamp_db = np.append(test_vamp_db, sig, axis = 0)
#                         test_on_events.append(time[ev])
    else:
        prev_mean = buffer.mean()
        buffer = np.array([])
        time_buffer = np.array([])
            
    time.sleep(0.1)            # wait (sleep) 0.1 seconds

3.00,2020-11-3 4:7:23.276

3.00,2020-11-3 4:7:23.304

3.00,2020-11-3 4:7:23.331

3.00,2020-11-3 4:7:23.359

3.00,2020-11-3 4:7:23.387

3.00,2020-11-3 4:7:23.415

3.00,2020-11-3 4:7:23.442

3.00,2020-11-3 4:7:23.470

3.00,2020-11-3 4:7:23.497

3.00,2020-11-3 4:7:23.525

3.00,2020-11-3 4:7:23.553

3.00,2020-11-3 4:7:23.580

3.00,2020-11-3 4:7:23.608

3.00,2020-11-3 4:7:23.636

3.00,2020-11-3 4:7:23.664

3.00,2020-11-3 4:7:23.692

3.00,2020-11-3 4:7:23.720

3.00,2020-11-3 4:7:23.747

3.00,2020-11-3 4:7:23.775

3.00,2020-11-3 4:7:23.803

3.00,2020-11-3 4:7:23.830

3.00,2020-11-3 4:7:23.858

3.00,2020-11-3 4:7:23.885

3.00,2020-11-3 4:7:23.913

3.00,2020-11-3 4:7:24.941

3.00,2020-11-3 4:7:24.969

3.00,2020-11-3 4:7:24.996

3.00,2020-11-3 4:7:24.24

3.00,2020-11-3 4:7:24.52

3.00,2020-11-3 4:7:24.79

3.00,2020-11-3 4:7:24.107

3.00,2020-11-3 4:7:24.135

3.00,2020-11-3 4:7:24.163

3.00,2020-11-3 4:7:24.190

3.00,2020-11-3 4:7:24.218

3.00,2020-11-3 4:7:24.246

3.00,2020-11-3 4:7:24.274

3.00


290.00,2020-11-3 4:8:5.741

290.00,2020-11-3 4:8:5.769

290.00,2020-11-3 4:8:5.796

290.00,2020-11-3 4:8:5.824

290.00,2020-11-3 4:8:5.852

290.00,2020-11-3 4:8:5.880

290.00,2020-11-3 4:8:5.908

290.00,2020-11-3 4:8:6.936

290.00,2020-11-3 4:8:6.963

290.00,2020-11-3 4:8:6.991

290.00,2020-11-3 4:8:6.19

290.00,2020-11-3 4:8:6.46

290.00,2020-11-3 4:8:6.74

290.00,2020-11-3 4:8:6.102

290.00,2020-11-3 4:8:6.130

290.00,2020-11-3 4:8:6.157

290.00,2020-11-3 4:8:6.185

290.00,2020-11-3 4:8:6.213

290.00,2020-11-3 4:8:6.241

290.00,2020-11-3 4:8:6.268

290.00,2020-11-3 4:8:6.296

290.00,2020-11-3 4:8:6.324

290.00,2020-11-3 4:8:6.352

290.00,2020-11-3 4:8:6.380

290.00,2020-11-3 4:8:6.407

290.00,2020-11-3 4:8:6.435

290.00,2020-11-3 4:8:6.463

290.00,2020-11-3 4:8:6.490

290.00,2020-11-3 4:8:6.518

290.00,2020-11-3 4:8:6.546

290.00,2020-11-3 4:8:6.573

290.00,2020-11-3 4:8:6.601

290.00,2020-11-3 4:8:6.629

290.00,2020-11-3 4:8:6.657

290.00,2020-11-3 4:8:6.684

290.00,2020-11-3 4:8:6

In [40]:
ser.close()

In [56]:
(pd.to_datetime('2020-11-03 00:28:50.575000'))> pd.to_datetime('2020-11-04 00:28:50.575000')

False

In [61]:
import datetime

In [69]:
pd.to_datetime('2020-11-03 00:28:50.575000') + datetime.timedelta(seconds=3)

Timestamp('2020-11-03 00:28:53.575000')